# Build connection, send data to database

In [ ]:
from IPython.display import Image
from IPython.core.display import HTML 
Image(url= "https://gcn.com/articles/2017/12/05/-/media/GIG/GCN/Redesign/Articles/2017/December/flightdata.png", width = 800)

**In this notebook, we will**
1. Set up a connection to a SQL database 
2. Download a csv file
3. Prepare the data for further processing
4. Push the prepared data to a table in the database
5. Use SQL to query data from the database



In [ ]:
# Import all necessary libraries
import pandas as pd
import numpy as np
import requests
from zipfile import *
from sqlalchemy import exc #SQLAlchemy provides a nice “Pythonic” way of interacting with databases.
from sqlalchemy import event

We will go through this workflow step by step before we include all single steps into one, big, main function which does all the steps at once.  
For our puropse, we will use public **flight data**. 

# 1. Set up a connection to a SQL database

We start with connecting to an existing sql database, so that we can check what is already there and to send data to a table in the database later on.

In [ ]:
# Establish db connection
from sql import conn
# Use code from data sourcing notebooks


# 2. Download csv file

In the following, you are going to download a csv file containing additional flight data from [this website](https://transtats.bts.gov).    
You can specify, which data you want to download.  
We want you to choose a specific month in a specific year for your download.  
In order to avoid everybody's downloading the same data, first check which months are already available in the flights table:

In [ ]:
#First, check which months are not in the database yet.
months = engine.execute('select distinct(year, month) from flights').fetchall()
months

Now, choose a month/year that is not yet in the database.  
With the following command lines, you will download a csv file on public flight data from [this website](https://transtats.bts.gov) containing data of your chosen month/year.    
The file will be stored in a data folder.

In [ ]:
years = [2018] # list of years you want to look at, specify one year
months = [9] # list of months you want to look at, specify one month
# Here: September 2018 which is in the list already
path ='data/' # Specifies path for saving file

# Loop through months
for year in years:
    for month in months:
        # Get the file from the website https://transtats.bts.gov
        zip_file = f'On_Time_Reporting_Carrier_On_Time_Performance_1987_present_{year}_{month}.zip'
        csv_file = f'On_Time_Reporting_Carrier_On_Time_Performance_(1987_present)_{year}_{month}.csv'
        url = (f'https://transtats.bts.gov/PREZIP/{zip_file}')
        arg = f' -P {path} --no-check-certificate'
        # Use the wget Module to download the file. The method accepts two parameters: the URL path of the file to download and local path where the file is to be stored.
        !wget {url}{arg} 

In [ ]:
# Unzip your file
with ZipFile(path+zip_file, 'r') as zip_ref:
    zip_ref.extractall(path)
    
# In casee this does not work for you try:
# Instead of 'from zipfile import *' use 'import zipfile' and use 'with zipfile.ZipFile(path+zip_file, 'r')'

In [ ]:
# Read in your data
df = pd.read_csv(path+csv_file, low_memory = False)
display(df.shape)
display(df.head())

# 3. Prepare the csv file for further processing

In the next step, we clean and prepare our dataset.

a) Since the dataset consists of a lot of columns, we we define which ones to keep.

In [ ]:
# Columns from downloaded file that are to be kept

columns_to_keep = [
                'FlightDate',
                'DepTime',
                'CRSDepTime',
                'DepDelay',
                'ArrTime',
                'CRSArrTime',
                'ArrDelay',
                'Reporting_Airline',
                'Tail_Number',
                'Flight_Number_Reporting_Airline',
                'Origin',
                'Dest',
                'AirTime',
                'Distance',
                'Cancelled',
                'Diverted'
]

In [ ]:
# The columns in the DB have different naming as in the source csv files. Lets get the names from the DB
table_name_sql = "SELECT COLUMN_NAME FROM INFORMATION_SCHEMA.COLUMNS WHERE TABLE_NAME = 'flights' ORDER BY ordinal_position"
c_names = engine.execute(table_name_sql).fetchall()
c_names

In [ ]:
# Just in case the above fails
c_names_alternate = ['flight_date', 'dep_time', 'sched_dep_time', 'dep_delay', 'arr_time', 'sched_arr_time', 
                'arr_delay', 'airline', 'tail_number', 'flight_number', 'origin', 'dest', 'air_time', 'distance', 'cancelled', 'diverted' ]

In [ ]:
# we can clean up the results into a clean list
new_column_names=[]
for name in c_names:
    new_column_names.append(name[0])
new_column_names        

b) With the next function, we make our csv file ready to be uploaded to SQL.  
We only keep to above specified columns and convert the datatypes.

In [ ]:
def clean_airline_df(df):
    '''
    Transforms a df made from BTS csv file into a df that is ready to be uploaded to SQL
    Set rows=0 for no filtering
    '''

    # Build dataframe including only the columns you want to keep
    df_airline = df.loc[:,columns_to_keep]
     
    # Clean data types and NULLs
    df_airline['FlightDate']= pd.to_datetime(df_airline['FlightDate'], yearfirst=True)
    df_airline['CRSArrTime']= pd.to_numeric(df_airline['CRSArrTime'], downcast='integer', errors='coerce')
    df_airline['Cancelled']= pd.to_numeric(df_airline['Cancelled'], downcast='integer')
    df_airline['Diverted']= pd.to_numeric(df_airline['Diverted'], downcast='integer')
    
    # Rename columns
    df_airline.columns = new_column_names
    
    return df_airline

In [ ]:
# Call function and check resulting dataframe
df_clean = clean_airline_df(df)
df_clean.head()

If you decide to only look at specific airports, it is a good decision to filter for them in advance.  
This function does the filtering. 

In [ ]:
# Specify the airports you are interested in and put them as a list in the function.
def select_airport(df, airports):
    ''' Helper function for filtering airline df for a subset of airports'''
    df_out = df.loc[(df.origin.isin(airports)) | (df.dest.isin(airports))]
    return df_out

In [ ]:
# Execute function, filtering for New York area airports
airports=['JFK', 'LGA', 'EWR']
if len(airports) > 0:
    df_airline = select_airport(df_clean, airports)
else:
    df_airline = df_clean
    
df_airline.head()

# 4. Push the prepared data to a table in the database

In [ ]:
# Specify which table within your database you want to push your data to. For our case, use the nyflights table.
# If the specified table doesn't exist yet, it will be created
# With 'append', your data will be appended to the already existing data within the table nyflights.
# This will take a minute or two...

table = 'flights'

# Sends your data to specified table, via the etablished connection
df_airline.to_sql(table, engine, index=False, if_exists="append", 
    method='multi', chunksize=5000)
print(f'done uploading {year}-{month}')

# 5. Use SQL to query data from the database

### Tasks
Having sent the data to the database, it is always good to check the results.

1. Check the top 5 rows of the nyflights table (as we did above)

2. Check if your data has been added by selecting your month and year.

3. See how many rows you have sent to the database by selecting your month and year in combination with count and group by.

# 6. Wrap-up

The following function puts everything we did above together.  

In [ ]:
def airline_csv_to_sql(years=[2020], months=[2], path ='data/', table='nyflights', airports=['JFK', 'LGA', 'EWR']):
    '''Downloads and unzips the flight data from BTS. Then Sends to SQL Database'''
    
    # Establish db connection
    params = config(section='postgres')
    engine = pg_engine_connection(**params)
    engine.dispose()

    # Loop through months
    for year in years:
        for month in months:
            # Get the file
            zip_file = f'On_Time_Reporting_Carrier_On_Time_Performance_1987_present_{year}_{month}.zip'
            csv_file = f'On_Time_Reporting_Carrier_On_Time_Performance_(1987_present)_{year}_{month}.csv'
            url = (f'https://transtats.bts.gov/PREZIP/{zip_file}')
            arg = f' -P {path} --no-check-certificate'
            !wget {url}{arg} 

            # unzip
            with ZipFile(path+zip_file, 'r') as zip_ref:
                zip_ref.extractall(path)
            
            # prepare df
            df_airline = pd.read_csv(path+csv_file, low_memory=False)
            df_airline = clean_airline_df(df_airline)
            
            # Select specific airports 
            if len(airports) > 0:
                df_airline = select_airport(df_airline, airports)

            # to SQL
            print(f'starting uploading {df_airline.shape[0]} rows from {year}-{month}')
            df_airline.to_sql(table, engine, index=False, if_exists="append", 
                  method='multi', chunksize=5000)
            print(f'done uploading {year}-{month}')